In [3]:
#basics of building recommendation systme
#that is a similiar to surprise.prediction_algorithms.preditions.Prediction
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate

class myownalgo(AlgoBase):
    def __init__(self):

        #Always call base method before doing anything
        AlgoBase.__init__(self)
    def estimate(self, u,i):
        details = {'info1':'that was', 'info2':'Easy stuff'}
        return 3, details

data = Dataset.load_builtin()#default ml-100k
algo = myownalgo()

cross_validate(algo,data, verbose=True)



Evaluating RMSE, MAE of algorithm myownalgo on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2486  1.2377  1.2339  1.2553  1.2451  1.2441  0.0077  
MAE (testset)     1.0066  0.9921  0.9909  1.0146  1.0041  1.0017  0.0090  
Fit time          0.00    0.02    0.02    0.02    0.02    0.02    0.01    
Test time         0.04    0.04    0.04    0.09    0.04    0.05    0.02    


{'test_rmse': array([1.24861924, 1.23767928, 1.23387601, 1.25532864, 1.24507028]),
 'test_mae': array([1.00655, 0.99215, 0.99095, 1.01455, 1.0041 ]),
 'fit_time': (0.0,
  0.018953323364257812,
  0.01891469955444336,
  0.01994609832763672,
  0.018933534622192383),
 'test_time': (0.038897037506103516,
  0.03590703010559082,
  0.03593897819519043,
  0.08978652954101562,
  0.03591442108154297)}

In [5]:
algo.estimate(3,1)

(3, {'info1': 'that was', 'info2': 'Easy stuff'})

In [6]:
#Fit Method
import numpy as np
class myownalgo2(AlgoBase):
    def __init__(self):
        #always call base method first
        AlgoBase.__init__(self)
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        #compute the average rating. we might as well use the
        # trainset.global_mean attribute
        #ignore the for's indexes just return the rating
        self.the_mean = np.mean([r for (_, _, r) in self.trainset.all_ratings()])
        #return self means out methods returns a reference to the instance object on which it was called
        # it allows us to use fluent and cascading specifications
        # like classmethod1().classmethod2().....classmethodn()
        # or like this algo.fit(trainset).test(testset)algo.fit(trainset).test(testset)
        return self
    def estimate(self, u,i):
        #trainset attribute
        sum_means = self.trainset.global_mean
        div = 1

        if self.trainset.knows_user(u):
            sum_means += np.mean([r for (_, r) in self.trainset.ur[u]])
            div +=1
        if self.trainset.knows_item(i):
            sum_means += np.mean([r for(_,r) in self.trainset.ir[i]])
            div += 1
            return sum/div    
        #return self.the_mean
#Note: the fit method called by cross_validate

In [7]:
#using baselines and similarities

class myownalgo3(AlgoBase):
    def __init__(self, sim_options={}, bsl_options={}):
        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)
        def fit(self, trainset):
            AlgoBase.fit(self, trainset)

            #compute the baselines and simirilarites
            self.bu, self.bi = self.compute_baselines()
            self.sim = self.compute_similarities()

            return self
    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown')
        #compute similarities between u and v, where v describes all other
        # users that have also rated item i
        neighbors = [(v, self.sim[u, v]) for(v, r) in self.trainset.ir[i]]
        # sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)
        print('the 3 nearset neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

In [ ]:
algo = myownalgo3()